<a href="https://colab.research.google.com/github/DJCordhose/deep-learning-crash-course-notebooks/blob/master/U4-M4-tf-prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing our Keras model for serving

In [1]:
import pandas as pd
print(pd.__version__)

0.25.3


In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

1.15.0


In [3]:
# let's see what compute devices we have available, hopefully a GPU 
sess = tf.Session()
devices = sess.list_devices()
for d in devices:
    print(d.name)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:XLA_CPU:0


In [4]:
# a small sane check, does tf seem to work ok?
hello = tf.constant('Hello TF!')
print(sess.run(hello))

b'Hello TF!'


In [5]:
from tensorflow import keras
print(keras.__version__)

2.2.4-tf


## Loading and validating our model

In [6]:
!curl -O https://raw.githubusercontent.com/DJCordhose/deep-learning-crash-course-notebooks/master/model/insurance.h5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  175k  100  175k    0     0  1218k      0 --:--:-- --:--:-- --:--:-- 1218k


In [7]:
!ls -l insurance.h5

-rw-r--r-- 1 root root 179704 Mar 21 15:48 insurance.h5


In [8]:
model = keras.models.load_model('./insurance.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Descison boundaries for 2 dimensions

![Descison Boundaries for 2 Dimensions](https://djcordhose.github.io/ai/img/manning/nn-reg.png)

In [9]:
# a little sanity check, does it work at all?

# within this code, we expect Olli to be a green customer with a high prabability
# 0: red
# 1: green
# 2: yellow

import numpy as np

olli_data = [100, 47, 10]

X = np.array([olli_data])
model.predict(X)

array([[0.00259071, 0.8607016 , 0.1367076 ]], dtype=float32)

## Converting our Keras model to the alternative high-level Estimator model

In [10]:
# https://cloud.google.com/blog/products/gcp/new-in-tensorflow-14-converting-a-keras-model-to-a-tensorflow-estimator
estimator_model = tf.keras.estimator.model_to_estimator(keras_model=model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmph6tstskk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f360c5d0be0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
# it still works the same, with a different style of API, though
x = {"hidden1_input": X}
list(estimator_model.predict(input_fn=tf.estimator.inputs.numpy_input_fn(x, shuffle=False)))

INFO:tensorflow:Could not find trained model in model_dir: /tmp/tmph6tstskk, running initialization to predict.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmph6tstskk/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: /tmp/tmph6tstskk/keras/keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 10 variables.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:

[{'softmax': array([1.3424826e-05, 0.0000000e+00, 9.9998653e-01], dtype=float32)}]

## Preparing our model for serving
* https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md
* https://www.tensorflow.org/serving/serving_basic

In [12]:
!rm -rf tf

import os

export_path_base = 'tf'
version = 1
export_path = os.path.join(
      tf.compat.as_bytes(export_path_base),
      tf.compat.as_bytes(str(version)))

tf.keras.backend.set_learning_phase(0)
sess = tf.keras.backend.get_session()

classification_inputs = tf.saved_model.utils.build_tensor_info(model.input)
classification_outputs_scores = tf.saved_model.utils.build_tensor_info(model.output)

signature =  tf.saved_model.signature_def_utils.build_signature_def(
    inputs={'inputs': classification_inputs},
    outputs={'scores': classification_outputs_scores},
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)

builder = tf.saved_model.builder.SavedModelBuilder(export_path)
builder.add_meta_graph_and_variables(
      sess, [tf.saved_model.tag_constants.SERVING],
      signature_def_map={
           tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature
      })
builder.save()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: tf/1/saved_model.pb


b'tf/1/saved_model.pb'